In [63]:
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
import pathlib
import re
from sklearn.model_selection import train_test_split

print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))

# class_names = {"A":0,"B":1,"C":2,"D":3,"E":4,"F":5,"G":6,"H":7,"I":8,"J":9,"K":10,"L":11,"M":12,"N":13,"O":14,"P":15,"Q":16,"R":17,"S":18,"T":19,"U":20,"V":21,"W":22,"X":23,"Y":24,"Z":25}
model_fp = "C:/Users/david/Documents/David/MSU/CSC450/imagenet_model_v1.h5"
# test_fp = "C:\\Users\\david\\Documents\\David\\MSU\\CSC450\\new_test_dataset.csv"
# train_fp = "C:\\Users\\david\\Documents\\David\\MSU\\CSC450\\new_train_dataset.csv"
data_dir = "C:/Users/david/Documents/David/MSU/CSC450/modified_dataset"

TensorFlow version: 2.4.0-dev20200902
Eager execution: True


In [74]:
def createDataset():
    dataset = tf.data.experimental.make_csv_dataset(
        file_pattern=data_dir+"/*.csv",
        column_names=["x","y","z"],
        header=False,
        batch_size=21,
        num_epochs = 1,
        shuffle_buffer_size=10000)
    return dataset

In [3]:
def pack_features_vector(features, labels):
  """Pack the features into a single array."""
  features = tf.stack(list(features.values()), axis=1)
  return features, labels

In [4]:
def create(num_classes):
    model = keras.applications.InceptionV3(
        include_top=True, weights='imagenet', input_tensor=None, input_shape=(21,3),
        pooling=None, classes=num_classes, classifier_activation='softmax'
    )
    model.compile(optimizer='adam',
        loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.metrics.SparseCategoricalAccuracy()])
    return model

In [73]:
ds = createDataset()
for feature_batch in ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of x:', feature_batch['x'])
for label_batch in ds.take(1):
  print('A batch of targets:', label_batch)

Every feature: ['x', 'y', 'z']
A batch of x: tf.Tensor(
[0.501465 0.397949 0.301025 0.223389 0.453857 0.563477 0.324707 0.772949
 0.631348 0.327637 0.336426 0.279541 0.512207 0.487061 0.290039 0.376221
 0.632812 0.563477 0.331299 0.489258 0.494385], shape=(21,), dtype=float32)
A batch of targets: OrderedDict([('x', <tf.Tensor: shape=(21,), dtype=float32, numpy=
array([0.28418 , 0.367432, 0.244263, 0.484375, 0.488525, 0.582031,
       0.679688, 0.66748 , 0.60791 , 0.43457 , 0.249756, 0.477051,
       0.430176, 0.532227, 0.495605, 0.563965, 0.395752, 0.608887,
       0.518555, 0.262451, 0.624512], dtype=float32)>), ('y', <tf.Tensor: shape=(21,), dtype=float32, numpy=
array([0.631836, 0.504883, 0.662598, 0.556152, 0.814941, 0.392822,
       0.697754, 0.693359, 0.422852, 0.632324, 0.414795, 0.10614 ,
       0.487549, 0.584473, 0.169312, 0.349365, 0.543457, 0.496826,
       0.325195, 0.702148, 0.544922], dtype=float32)>), ('z', <tf.Tensor: shape=(21,), dtype=float32, numpy=
array([-4.62646e

In [15]:
model = create()
history = model.fit(train_ds, epochs=100)
loss, acc = model.evaluate(test_ds, verbose=2)
print("Model, accuracy: {:5.2f}%".format(100*acc))
print("Model, loss: {:5.2f}%".format(100*loss))
model.save(model_fp)

Epoch 1/100
203/203 [==============================] - 2s 11ms/step - loss: 3.2071 - sparse_categorical_accuracy: 0.1357
Epoch 2/100
203/203 [==============================] - 1s 4ms/step - loss: 3.1019 - sparse_categorical_accuracy: 0.2422
Epoch 3/100
203/203 [==============================] - 1s 4ms/step - loss: 3.0388 - sparse_categorical_accuracy: 0.3043
Epoch 4/100
203/203 [==============================] - 1s 3ms/step - loss: 3.0077 - sparse_categorical_accuracy: 0.3274
Epoch 5/100
203/203 [==============================] - 1s 4ms/step - loss: 2.9880 - sparse_categorical_accuracy: 0.3461
Epoch 6/100
203/203 [==============================] - 1s 4ms/step - loss: 2.9723 - sparse_categorical_accuracy: 0.3608
Epoch 7/100
203/203 [==============================] - 1s 3ms/step - loss: 2.9548 - sparse_categorical_accuracy: 0.3815
Epoch 8/100
203/203 [==============================] - 1s 4ms/step - loss: 2.9428 - sparse_categorical_accuracy: 0.3916
Epoch 9/100
203/203 [==================

In [16]:
new_model = tf.keras.models.load_model(model_fp)
new_model.summary()
predictions = new_model.predict(test_ds)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                4096      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 26)                1690      
Total params: 9,946
Trainable params: 9,946
Non-trainable params: 0
_________________________________________________________________


In [19]:
df = pd.read_csv(test_fp)
count = 0
correct = 0
total = len(df)
for pred in predictions:
    actual = df.loc[count,"label"]
    prediction = np.argmax(pred)
    # print("Actual:", actual, "Prediction:", prediction)
    if actual == prediction:
        correct += 1
    count += 1
print("--------------------------\n"+str(correct)+" out of "+str(total)+" predicted correctly")
print("Accuracy: "+str((correct/total)*100)+"%")

--------------------------
1577 out of 38304 predicted correctly
Accuracy: 4.117063492063492%
